In [1]:
import sys,os,argparse,time
import numpy as np
import torch
import utils
from datetime import datetime
import pickle
tstart=time.time()

In [2]:
# Arguments
parser=argparse.ArgumentParser(description='xxx')
parser.add_argument('--seed',               default=0,              type=int,     help='(default=%(default)d)')
parser.add_argument('--device',             default='cpu',          type=str,     help='gpu id')
parser.add_argument('--experiment',         default='16_task_groups',       type =str,    help='Mnist or dissertation')
parser.add_argument('--approach',           default='PUGCL',          type =str,    help='Method, always Lifelong Uncertainty-aware learning')
parser.add_argument('--data_path',          default='data/data.csv',     type=str,     help='gpu id')

# Training parameters
parser.add_argument('--output',             default='',             type=str,     help='')
parser.add_argument('--checkpoint_dir',     default='../checkpoints_16_tasks',    type=str,   help='')
parser.add_argument('--n_epochs',           default=100,              type=int,     help='')
parser.add_argument('--batch_size',         default=64,             type=int,     help='')
parser.add_argument('--lr',                 default=0.01,           type=float,   help='')
parser.add_argument('--hidden_size',        default=800,           type=int,     help='')
parser.add_argument('--parameter',          default='',             type=str,     help='')

# UCB HYPER-PARAMETERS
parser.add_argument('--MC_samples',         default='10',           type=int,     help='Number of Monte Carlo samples')
parser.add_argument('--rho',                default='-3',           type=float,   help='Initial rho')
parser.add_argument('--sigma1',             default='0.0',          type=float,   help='STD foor the 1st prior pdf in scaled mixture Gaussian')
parser.add_argument('--sigma2',             default='6.0',          type=float,   help='STD foor the 2nd prior pdf in scaled mixture Gaussian')
parser.add_argument('--pi',                 default='0.25',         type=float,   help='weighting factor for prior')

parser.add_argument('--resume',             default='no',           type=str,     help='resume?')
parser.add_argument('--sti',                default=0,              type=int,     help='starting task?')

parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")

args=parser.parse_args()
utils.print_arguments(args)

Arguments =
	seed: 0
	device: cpu
	experiment: 16_task_groups
	approach: PUGCL
	data_path: data/data.csv
	output: 
	checkpoint_dir: ../checkpoints_16_tasks
	n_epochs: 100
	batch_size: 64
	lr: 0.01
	hidden_size: 800
	parameter: 
	MC_samples: 10
	rho: -3.0
	sigma1: 0.0
	sigma2: 6.0
	pi: 0.25
	resume: no
	sti: 0
	fff: /Users/jonastjomsland/Library/Jupyter/runtime/kernel-003dd4ae-5b1e-4b75-9dcd-182fa5aa8112.json


In [3]:
# Set seed for stable results
np.random.seed(args.seed)
torch.manual_seed(args.seed)

# Check if Cuda is available
if torch.cuda.is_available():
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print("Using device:", args.device)

# Checkpoint
checkpoint = utils.make_directories(args)
args.checkpoint = checkpoint
print()

# PUGCL with two tasks:
from data import dataloader_16_tasks as dataloader

# Import Lifelong Uncertainty-aware Learning approach:
#from bayesian_model.lul import Lul
from training_method import PUGCL

# Import model used:
#from bayesian_model.bayesian_network import BayesianNetwork
from bayesian_model.bayesian_network import BayesianNetwork

Using device: cpu
16_task_groups_PUGCL
Results will be saved in  ../checkpoints_16_tasks/16_task_groups_PUGCL



In [4]:
print()
print("Starting this session on: ")
print(datetime.now().strftime("%Y-%m-%d %H:%M"))

# Load data:
print("Loading data...")
data, task_outputs, input_size = dataloader.get(data_path=args.data_path)
print("Input size =", input_size, "\nTask info =", task_outputs)
print("Number of data samples: ", len(data[0]['train']['x']))
args.num_tasks = len(task_outputs)
args.input_size = input_size
args.task_outputs = task_outputs
pickle.dump(data, open( "data/data.p", "wb" ))

# Initialize Bayesian network
print("Initializing network...")
model = BayesianNetwork(args).to(args.device)

# Initialize Lul approach
print("Initialize Lifelong Uncertainty-aware Learning")
approach = PUGCL(model, args=args)
print("-"*100)

# Check wether resuming:
if args.resume == "yes":
    checkpoint = torch.load(os.path.join(args.checkpoint, 'model_{}.pth.tar'.format(args.sti)))
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device=args.device)
else:
    args.sti = 0


Starting this session on: 
2020-05-17 08:08
Loading data...
Input size = [1, 29] 
Task info = [(0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (15, 2)]
Number of data samples:  500
Initializing network...
Initialize Lifelong Uncertainty-aware Learning
----------------------------------------------------------------------------------------------------


/Users/jonastjomsland/anaconda3/envs/ucb/lib/python3.6/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [5]:
# Iterate over the two tasks:
loss = np.zeros((len(task_outputs), len(task_outputs)), dtype=np.float32)
for task, n_class in task_outputs[args.sti:]:
    print('*'*100)
    print('Task {:2d} ({:s})'.format(task, data[task]['name']))
    print('*'*100)

    # Get data:
    xtrain = data[task]['train']['x'][:,1:].type(torch.float32).to(args.device)
    ytrain = data[task]['train']['y'].type(torch.float32).to(args.device)
    xvalid = data[task]['valid']['x'][:,1:].type(torch.float32).to(args.device)
    yvalid = data[task]['valid']['y'].type(torch.float32).to(args.device)

    # Start training
    print("Starting training for the tasks in group: ", task)
    approach.train(task, xtrain, ytrain, xvalid, yvalid)
    print('_'*100)

    # Validate for this task group:
    for u in range(task+1):
        xtest = data[u]['test']['x'][:,1:].type(torch.float32).to(args.device)
        ytest = data[u]['test']['y'].type(torch.float32).to(args.device)
        test_loss = approach.eval(u, xtest, ytest, debug=True)
        print("Test on task {:2d} - {:15s}: loss={:.3f}".format(u, data[u]['name'], test_loss))
        loss[task, u] = test_loss

    # Save
    print("Saving at " + args.checkpoint)
    np.savetxt(os.path.join(args.checkpoint, '{}_{}_{}.txt'.format(args.experiment, args.approach, args.seed)), loss, '%.5f')

****************************************************************************************************
Task  0 (Vacuum cleaning)
****************************************************************************************************
Starting training for the tasks in group:  0
Batch: 0/500 

../torch/csrc/utils/python_arg_parser.cpp:750: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, Number alpha)


Batch: 448/500 | Epoch   1, time=668.0ms/ 16.3ms | Training loss: 1.939 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch   2, time=658.0ms/ 18.1ms | Training loss: 1.825 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch   3, time=1077.4ms/157.2ms | Training loss: 1.778 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch   4, time=861.6ms/ 16.5ms | Training loss: 1.741 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch   5, time=704.9ms/ 18.2ms | Training loss: 1.700 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch   6, time=714.4ms/ 19.1ms | Training loss: 1.803 | Learning rate: 0.010 |
Batch: 448/500 | Epoch   7, time=701.1ms/ 19.9ms | Training loss: 1.710 | Learning rate: 0.010 |
Batch: 448/500 | Epoch   8, time=692.9ms/ 18.0ms | Training loss: 1.725 | Learning rate: 0.010 |
Batch: 448/500 | Epoch   9, time=718.2ms/ 19.9ms | Training loss: 1.631 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  10, time=667.4ms/ 15.9ms | Training loss: 1.704 | Learning rate: 0.010 |
Batch: 448/500 | 

Batch: 448/500 | Epoch  85, time=598.1ms/ 15.2ms | Training loss: 1.482 | Learning rate: 0.000 | lr=4.6e-06
Batch: 448/500 | Epoch  86, time=601.9ms/ 15.4ms | Training loss: 1.458 | Learning rate: 0.000 | *
Batch: 448/500 | Epoch  87, time=603.1ms/ 15.0ms | Training loss: 1.454 | Learning rate: 0.000 | *
Batch: 448/500 | Epoch  88, time=603.9ms/ 14.3ms | Training loss: 1.455 | Learning rate: 0.000 |
Batch: 448/500 | Epoch  89, time=619.4ms/ 16.6ms | Training loss: 1.480 | Learning rate: 0.000 |
Batch: 448/500 | Epoch  90, time=606.6ms/ 16.8ms | Training loss: 1.473 | Learning rate: 0.000 |
Batch: 448/500 | Epoch  91, time=601.7ms/ 15.7ms | Training loss: 1.452 | Learning rate: 0.000 | *
Batch: 448/500 | Epoch  92, time=605.7ms/ 15.6ms | Training loss: 1.505 | Learning rate: 0.000 |
Batch: 448/500 | Epoch  93, time=610.1ms/ 15.5ms | Training loss: 1.474 | Learning rate: 0.000 |
Batch: 448/500 | Epoch  94, time=616.8ms/ 15.5ms | Training loss: 1.500 | Learning rate: 0.000 |
Batch: 448/50

Batch: 448/500 | Epoch  63, time=614.3ms/ 15.4ms | Training loss: 2.138 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  64, time=609.2ms/ 15.6ms | Training loss: 2.119 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  65, time=611.7ms/ 16.0ms | Training loss: 2.108 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  66, time=610.5ms/ 15.1ms | Training loss: 2.095 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  67, time=614.4ms/ 16.0ms | Training loss: 2.092 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  68, time=614.6ms/ 15.5ms | Training loss: 2.087 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  69, time=603.4ms/ 15.4ms | Training loss: 2.079 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  70, time=611.7ms/ 15.5ms | Training loss: 2.056 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  71, time=617.7ms/ 14.9ms | Training loss: 2.046 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  72, time=608.9ms/ 15.6ms | Training loss: 2.020 | Learning rate: 0.010 | *
Batch: 448

...

Batch: 448/500 | Epoch  42, time=601.4ms/ 15.8ms | Training loss: 1.853 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  43, time=605.3ms/ 15.4ms | Training loss: 1.837 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  44, time=609.6ms/ 15.6ms | Training loss: 1.818 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  45, time=600.0ms/ 15.8ms | Training loss: 1.802 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  46, time=611.9ms/ 16.2ms | Training loss: 1.787 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  47, time=628.3ms/ 15.5ms | Training loss: 1.770 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  48, time=611.7ms/ 15.4ms | Training loss: 1.756 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  49, time=605.9ms/ 14.9ms | Training loss: 1.740 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  50, time=605.4ms/ 14.2ms | Training loss: 1.726 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  51, time=609.0ms/ 15.3ms | Training loss: 1.712 | Learning rate: 0.010 | *
Batch: 448

Batch: 448/500 | Epoch  14, time=607.8ms/ 15.8ms | Training loss: 2.265 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  15, time=610.6ms/ 15.2ms | Training loss: 2.241 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  16, time=607.6ms/ 15.4ms | Training loss: 2.217 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  17, time=610.2ms/ 15.6ms | Training loss: 2.193 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  18, time=610.0ms/ 15.4ms | Training loss: 2.169 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  19, time=607.1ms/ 16.1ms | Training loss: 2.142 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  20, time=608.1ms/ 15.5ms | Training loss: 2.117 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  21, time=610.1ms/ 16.1ms | Training loss: 2.097 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  22, time=609.9ms/ 15.3ms | Training loss: 2.074 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  23, time=610.9ms/ 15.8ms | Training loss: 2.055 | Learning rate: 0.010 | *
Batch: 448

Batch: 448/500 | Epoch  97, time=612.0ms/ 15.5ms | Training loss: 1.608 | Learning rate: 0.010 |
Batch: 448/500 | Epoch  98, time=607.2ms/ 15.6ms | Training loss: 1.606 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  99, time=611.8ms/ 15.3ms | Training loss: 1.605 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch 100, time=604.4ms/ 14.3ms | Training loss: 1.601 | Learning rate: 0.010 | *
____________________________________________________________________________________________________
Test on task  0 - Vacuum cleaning: loss=1.576
Test on task  1 - Mopping the floor: loss=2.870
Test on task  2 - Carry warm food: loss=1.760
Test on task  3 - Carry cold food: loss=1.780
Test on task  4 - Carry drinks   : loss=1.500
Test on task  5 - Carry small objects (plates, toys): loss=1.457
Test on task  6 - Carry big objects (tables, chairs): loss=1.575
Test on task  7 - Cleaning (Picking up stuff): loss=1.732
Test on task  8 - Vacuum cleaning: loss=2.015
Test on task  9 - Mopping the floor: lo

Batch: 448/500 | Epoch  68, time=608.7ms/ 15.3ms | Training loss: 1.743 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  69, time=609.2ms/ 15.2ms | Training loss: 1.740 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  70, time=603.7ms/ 16.0ms | Training loss: 1.737 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  71, time=606.0ms/ 14.9ms | Training loss: 1.735 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  72, time=602.6ms/ 15.3ms | Training loss: 1.731 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  73, time=607.3ms/ 14.5ms | Training loss: 1.729 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  74, time=603.9ms/ 15.4ms | Training loss: 1.726 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  75, time=599.2ms/ 15.1ms | Training loss: 1.724 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  76, time=605.2ms/ 15.1ms | Training loss: 1.721 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  77, time=603.4ms/ 15.6ms | Training loss: 1.720 | Learning rate: 0.010 | *
Batch: 448

Batch: 448/500 | Epoch  39, time=611.2ms/ 15.5ms | Training loss: 1.981 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  40, time=612.7ms/ 15.6ms | Training loss: 1.971 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  41, time=610.8ms/ 15.6ms | Training loss: 1.963 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  42, time=609.7ms/ 16.2ms | Training loss: 1.947 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  43, time=606.1ms/ 16.1ms | Training loss: 1.936 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  44, time=610.2ms/ 15.9ms | Training loss: 1.926 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  45, time=599.6ms/ 15.6ms | Training loss: 1.916 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  46, time=625.8ms/ 17.5ms | Training loss: 1.904 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  47, time=614.6ms/ 16.8ms | Training loss: 1.895 | Learning rate: 0.010 | *
Batch: 448/500 | Epoch  48, time=627.2ms/ 15.4ms | Training loss: 1.884 | Learning rate: 0.010 | *
Batch: 448